In [1]:
from biological_fuzzy_logic_networks.DREAM_analysis.utils import (
    prepare_cell_line_data,
    cl_data_to_input,
)
from biological_fuzzy_logic_networks.DREAM.DREAMBioFuzzNet import DREAMBioFuzzNet
# import pandas as pd
# from typing import List, Union, Sequence
# from sklearn.metrics import r2_score
# import mlflow
# import click
import json
# import torch
# import pickle as pickle
# import os
import networkx as nx

/dccstor/ipc1/.conda/envs/biofuzznet2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu


In [2]:
with open("/dccstor/ipc1/CAR/DREAM/Model/Test/After_synthetic/MFM223_1/MFM223_config.json") as f:
    config = json.load(f)
f.close()
config["pkn_sif"] = "/dccstor/ipc1/CAR/DREAM/DREAMdata/Alice_subnetwork.sif"
del config["root_nodes"]
del config["add_root_values"]
del config["input_value"]

In [3]:
model = DREAMBioFuzzNet.build_DREAMBioFuzzNet_from_file(config["pkn_sif"], n=1.1, K=1)
cl_data = prepare_cell_line_data(
    data_file=config["data_file"],
    time_point=config["time_point"],
    non_marker_cols=config["non_marker_cols"],
    treatment_col_name=config["treatment_col_name"],
    filter_starved_stim=config["filter_starved_stim"],
    sample_n_cells=config["sample_n_cells"],
    sel_condition=config["sel_condition"],
)

<class 'str'>
['MFM223']


In [4]:
sorted(nx.simple_cycles(model))

[]

In [5]:
sorted(model.edges)

[('AKT', 'GSK3B'),
 ('AKT', 'or10'),
 ('AKT_S473', 'or12'),
 ('AKT_T308', 'or12'),
 ('AMPK', 'or2'),
 ('ERK12', 'MKK36'),
 ('ERK12', 'or4'),
 ('ERK12', 'or9'),
 ('ERK12', 'p90RSK'),
 ('FAK', 'ERK12'),
 ('GSK3B', 'or1'),
 ('MAPKAPK2', 'or11'),
 ('MKK36', 'or4'),
 ('MKK36', 'or7'),
 ('MKK4', 'JNK'),
 ('MKK4', 'or8'),
 ('MSK12', 'or6'),
 ('SMAD23', 'or3'),
 ('SRC', 'FAK'),
 ('SRC', 'or7'),
 ('or1', 'or2'),
 ('or10', 'or11'),
 ('or11', 'CREB'),
 ('or12', 'AKT'),
 ('or2', 'RB'),
 ('or3', 'cleavedCas'),
 ('or4', 'or5'),
 ('or5', 'MSK12'),
 ('or6', 'H3'),
 ('or7', 'or8'),
 ('or8', 'p38'),
 ('or9', 'MAPKAPK2'),
 ('p38', 'or5'),
 ('p38', 'or9'),
 ('p38', 'p53'),
 ('p53', 'or1'),
 ('p53', 'or3'),
 ('p90RSK', 'or10'),
 ('p90RSK', 'or6')]

In [6]:
nx.is_directed(model) == nx.is_directed_acyclic_graph(model)

True

In [7]:
"EGFR" in model.nodes # Should be false

False

In [8]:
model.root_nodes

['AMPK', 'SMAD23', 'AKT_S473', 'AKT_T308', 'SRC', 'MKK4']

In [9]:
model.biological_nodes

['p53',
 'RB',
 'GSK3B',
 'AMPK',
 'cleavedCas',
 'SMAD23',
 'ERK12',
 'MSK12',
 'MKK36',
 'H3',
 'p90RSK',
 'p38',
 'MAPKAPK2',
 'CREB',
 'AKT_S473',
 'AKT',
 'AKT_T308',
 'SRC',
 'FAK',
 'MKK4',
 'JNK']

In [10]:
# Load train and valid data
(
    train_data,
    valid_data,
    train_inhibitors,
    valid_inhibitors,
    train_input,
    valid_input,
    train,
    valid,
    scaler,
) = cl_data_to_input(
    data=cl_data,
    model=model,
    train_treatments=config["train_treatments"],
    valid_treatments=config["valid_treatments"],
    train_cell_lines=config["train_cell_lines"],
    valid_cell_lines=config["valid_cell_lines"],
    inhibition_value=config["inhibition_value"],
    replace_zero_inputs=config["replace_zero_inputs"],
    balance_data=True,
)

/u/adr/Code/biological_fuzzy_logic_networks/biological_fuzzy_logic_networks/DREAM_analysis/utils.py:305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t[t < 0] = 1e-9
/u/adr/Code/biological_fuzzy_logic_networks/biological_fuzzy_logic_networks/DREAM_analysis/utils.py:305: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t[t < 0] = 1e-9
/u/adr/Code/biological_fuzzy_logic_networks/biological_fuzzy_logic_networks/DREAM_analysis/utils.py:306: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer

In [12]:
loss, best_val_loss, loop_states = model.conduct_optimisation(
    input=train_input,
    valid_input=valid_input,
    ground_truth=train_data,
    valid_ground_truth=valid_data,
    train_inhibitors=train_inhibitors,
    valid_inhibitors=valid_inhibitors,
    epochs=3,
    learning_rate=config["learning_rate"],
    batch_size=config["batch_size"],
)



Loss:1.25e-01: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:16<00:00,  5.64s/it]


In [ ]:
loss


In [ ]:
best_val_loss